# BERT

### bert_en_UNCASED_L-24_H-1024_A-16

In [1]:
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import re
import numpy as np

input_file = 'test_data-CLEANED-1.csv' #'train_data-CLEANED-1.csv'
output_file = 'test_data-CLEANED-2.csv' #'train_data-CLEANED-2.csv'

In [2]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "bert_en_uncased_preprocess_3-next")
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer(
    "bert_en_uncased_L-24_H-1024_A-16_4",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 1024].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 1024].


In [3]:
# WARNING:tensorflow:Please fix your imports. Module tensorflow.python.training.tracking.data_structures has been moved to tensorflow.python.trackable.data_structures. The old module will be deleted in version 2.11.
# probably no need to worry about this warning

In [4]:
embedding_model = tf.keras.Model(text_input, pooled_output)

In [5]:
# embeddings can be created for both words and sentences
sentences = tf.constant(["Hello World", "Hello"])
embeddings = embedding_model(sentences)
print(embeddings.shape)
print(embeddings[0])

(2, 1024)
tf.Tensor(
[-0.9958025  -0.99153316  0.9999988  ... -0.9999684   0.98763025
 -0.9921667 ], shape=(1024,), dtype=float32)


### Embeddings for parts of affiliations

In [6]:
t_data = pd.read_csv(input_file, sep = ";")
t_data.drop(columns = ['Unnamed: 0'], inplace = True)
t_data

,part,labels,affil_index,embeddings_csd-BERT,embeddings_uncsd-BERT
0,Aventis Pharma,Institution,0,"[-0.6225269436836243, 0.41175007820129395, 0.9...","[-0.7495080828666687, 0.10450626909732819, 0.8..."
1,Romainville,City,0,"[-0.7443768978118896, 0.37689119577407837, 0.9...","[-0.9179261922836304, -0.47461554408073425, -0..."
2,France. christophe.dini@aventis.com,Country,0,"[-0.7713785767555237, 0.44986480474472046, 0.9...","[-0.8849691152572632, -0.5707671642303467, -0...."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[-0.615035891532898, 0.31155872344970703, 0.99...","[-0.9479542970657349, -0.6653308272361755, -0...."
4,China Pharmaceutical University,Institution,1,"[-0.738187313079834, 0.46809735894203186, 0.99...","[-0.9523708820343018, -0.5192119479179382, -0...."
...,...,...,...,...,...
311,People's Republic of China.,Country,58,"[-0.7836729288101196, 0.5700669884681702, 0.99...","[-0.9732127785682678, -0.5987682938575745, -0...."
312,National Center for Natural Products Research,Institution,59,"[-0.6381785273551941, 0.46710556745529175, 0.9...","[-0.9274004697799683, -0.2877633571624756, -0...."
313,The University of Mississippi,Institution,59,"[-0.676986813545227, 0.4199213683605194, 0.999...","[-0.9446776509284973, -0.3148469924926758, -0...."
314,MS 38677,State,59,"[-0.7675344944000244, 0.5155598521232605, 0.99...","[-0.919567883014679, -0.28708547353744507, -0...."


In [7]:
parts = t_data.iloc[:, 0].tolist()
parts

['Aventis Pharma',
 'Romainville',
 'France. christophe.dini@aventis.com',
 'Jiangsu Key Laboratory of Drug Design and Optimization',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China',
 'Key Laboratory on Protein Chemistry and Structural Biology',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China.',
 'Universidade Federal do Espírito Santo',
 'Rodovia BR 101 Norte',
 'Km 60',
 '29932-900',
 'Brazil.',
 'Jamia Hamdard',
 'New Delhi 110062',
 'India.',
 'Pondicherry University',
 'Puducherry 605 014',
 'India. Electronic address: tchinnasamy@hotmail.com.',
 'Payame Noor University',
 'Khoy 58168-45164',
 'Iran. saghatforoush@gmail.com',
 'Jadavpur University',
 'Kolkata 700 032',
 'India.',
 'Jagiellonian University Medical College',
 'Kraków',
 'Poland. Electronic address: mfmalaws@cyf-kr.edu.pl.',
 'Istituto Superiore di Sanità',
 'Viale Regina Elena 299',
 '00161 Roma',
 'Italy. carlo.mustazza@

In [8]:
#embeddings = embedding_model(tf.constant(parts))
#print(embeddings.shape)

In [9]:
embeddings_parts = []
for part in parts:
    try:
        embeddings_parts.append(embedding_model(tf.constant([part])).numpy().tolist())
    except:
        print(part)
        embeddings_parts.append("X")
t_data["embeddings_uncsd-BERT-1024"] = embeddings_parts
#took around 40 minutes to run for 3000 parts of 600 affils

In [10]:
t_data

,part,labels,affil_index,embeddings_csd-BERT,embeddings_uncsd-BERT,embeddings_uncsd-BERT-1024
0,Aventis Pharma,Institution,0,"[-0.6225269436836243, 0.41175007820129395, 0.9...","[-0.7495080828666687, 0.10450626909732819, 0.8...","[[-0.9385257363319397, -0.8636099100112915, 0...."
1,Romainville,City,0,"[-0.7443768978118896, 0.37689119577407837, 0.9...","[-0.9179261922836304, -0.47461554408073425, -0...","[[-0.9338648915290833, -0.8978722095489502, 0...."
2,France. christophe.dini@aventis.com,Country,0,"[-0.7713785767555237, 0.44986480474472046, 0.9...","[-0.8849691152572632, -0.5707671642303467, -0....","[[-0.925447940826416, -0.9710708260536194, 0.9..."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[-0.615035891532898, 0.31155872344970703, 0.99...","[-0.9479542970657349, -0.6653308272361755, -0....","[[-0.9993273019790649, -0.998293399810791, 0.9..."
4,China Pharmaceutical University,Institution,1,"[-0.738187313079834, 0.46809735894203186, 0.99...","[-0.9523708820343018, -0.5192119479179382, -0....","[[-0.9705463647842407, -0.9841862916946411, 0...."
...,...,...,...,...,...,...
311,People's Republic of China.,Country,58,"[-0.7836729288101196, 0.5700669884681702, 0.99...","[-0.9732127785682678, -0.5987682938575745, -0....","[[-0.8492422699928284, -0.9690293669700623, 0...."
312,National Center for Natural Products Research,Institution,59,"[-0.6381785273551941, 0.46710556745529175, 0.9...","[-0.9274004697799683, -0.2877633571624756, -0....","[[-0.9930930137634277, -0.9795610904693604, 0...."
313,The University of Mississippi,Institution,59,"[-0.676986813545227, 0.4199213683605194, 0.999...","[-0.9446776509284973, -0.3148469924926758, -0....","[[-0.10345132648944855, -0.3611591160297394, 0..."
314,MS 38677,State,59,"[-0.7675344944000244, 0.5155598521232605, 0.99...","[-0.919567883014679, -0.28708547353744507, -0....","[[-0.7636699676513672, -0.933850109577179, 0.9..."


In [11]:
t_data["embeddings_uncsd-BERT-1024"] = t_data["embeddings_uncsd-BERT-1024"].apply(lambda x: x[0])
t_data

,part,labels,affil_index,embeddings_csd-BERT,embeddings_uncsd-BERT,embeddings_uncsd-BERT-1024
0,Aventis Pharma,Institution,0,"[-0.6225269436836243, 0.41175007820129395, 0.9...","[-0.7495080828666687, 0.10450626909732819, 0.8...","[-0.9385257363319397, -0.8636099100112915, 0.9..."
1,Romainville,City,0,"[-0.7443768978118896, 0.37689119577407837, 0.9...","[-0.9179261922836304, -0.47461554408073425, -0...","[-0.9338648915290833, -0.8978722095489502, 0.9..."
2,France. christophe.dini@aventis.com,Country,0,"[-0.7713785767555237, 0.44986480474472046, 0.9...","[-0.8849691152572632, -0.5707671642303467, -0....","[-0.925447940826416, -0.9710708260536194, 0.99..."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[-0.615035891532898, 0.31155872344970703, 0.99...","[-0.9479542970657349, -0.6653308272361755, -0....","[-0.9993273019790649, -0.998293399810791, 0.99..."
4,China Pharmaceutical University,Institution,1,"[-0.738187313079834, 0.46809735894203186, 0.99...","[-0.9523708820343018, -0.5192119479179382, -0....","[-0.9705463647842407, -0.9841862916946411, 0.9..."
...,...,...,...,...,...,...
311,People's Republic of China.,Country,58,"[-0.7836729288101196, 0.5700669884681702, 0.99...","[-0.9732127785682678, -0.5987682938575745, -0....","[-0.8492422699928284, -0.9690293669700623, 0.9..."
312,National Center for Natural Products Research,Institution,59,"[-0.6381785273551941, 0.46710556745529175, 0.9...","[-0.9274004697799683, -0.2877633571624756, -0....","[-0.9930930137634277, -0.9795610904693604, 0.9..."
313,The University of Mississippi,Institution,59,"[-0.676986813545227, 0.4199213683605194, 0.999...","[-0.9446776509284973, -0.3148469924926758, -0....","[-0.10345132648944855, -0.3611591160297394, 0...."
314,MS 38677,State,59,"[-0.7675344944000244, 0.5155598521232605, 0.99...","[-0.919567883014679, -0.28708547353744507, -0....","[-0.7636699676513672, -0.933850109577179, 0.99..."


In [12]:
t_data.loc[0, "embeddings_uncsd-BERT-1024"]

[-0.9385257363319397,
 -0.8636099100112915,
 0.9926379323005676,
 -0.915038526058197,
 -0.9865367412567139,
 0.6873221397399902,
 -0.9990100264549255,
 0.9941737651824951,
 0.837611198425293,
 -0.8281686305999756,
 -0.9599183797836304,
 0.999439537525177,
 -0.9997760653495789,
 -0.06132330372929573,
 0.835709273815155,
 -0.9325147867202759,
 -0.9994634985923767,
 -0.9997303485870361,
 0.9882432818412781,
 0.9609547257423401,
 -0.8860683441162109,
 0.9929625988006592,
 -0.8843388557434082,
 -0.9846020340919495,
 0.5943376421928406,
 -0.9979391098022461,
 0.9988675713539124,
 -0.8849419951438904,
 0.15077801048755646,
 -0.8783887624740601,
 -0.42157498002052307,
 -0.9731826186180115,
 -0.9489157795906067,
 -0.9598912596702576,
 -0.9387302398681641,
 -0.9996380805969238,
 0.9998947978019714,
 0.9433959722518921,
 0.906663715839386,
 -0.9900919198989868,
 0.9963204860687256,
 -0.7311635613441467,
 -0.9999666213989258,
 0.965947151184082,
 0.9772759079933167,
 0.9994558691978455,
 0.9953798

In [13]:
t_data.to_csv(output_file, sep = ';')